## Homework 4, Problem 4 Classification on real data

ECE C143A/C243A, Spring Quarter 2018, Prof. J.C. Kao, TAs T. Monsoor, X. Jiang and X. Yang

## Background
Neural prosthetic systems can be built based on classifying neural activity related to planning. As described in class, this is analogous to mapping patterns of neural activity to keys on a keyboard.
In this problem, we will apply the results of Problems 1 and 2 to real neural data. The neural data were recorded using a 100-electrode array in premotor cortex of a macaque monkey1. The dataset can be found on CCLE as `ps4_realdata.mat`.

The following describes the data format. The `.mat` file is loaded into Python as a dictionary with two keys: `train_trial` contains the training data and `test_trial` contains the test data. Each of these contains spike trains recorded simultaneously from 97 neurons while the monkey reached 91 times along each of 8 different reaching angles.

The spike train recorded from the $i_{th}$ neuron on the $n_{th}$ trial of the $k_{th}$ reaching angle is accessed as 

`data['train_trial'][n,k][1][i,:]`

where n = 0,...,90, k = 0,...,7, and i = 0, . . . , 96.  The [1] in between [n,k] and [i,:] does not mean anything for this assignment and is simply an "artifact" of how the data is structured. A spike train is represented as a sequence of zeros and ones, where time is discretized in 1 ms steps. A zero indicates that the neuron did not spike in the 1 ms bin, whereas a one indicates that the neuron spiked once in the 1 ms bin. The structure test trial has the same format as train trial.

Each spike train is 700 ms long (and thus represented by an array of length 700).  This comprises a 200ms baseline period (before the reach target turned on), a 500ms planning period (after the reach target turned on).  Because it takes time for information about the reach target to arrive in premotor cortex (due to the time required for action potentials to propagate and for visual processing), we will ignore the first 150ms of the planning period.  *** FOR THIS PROBLEM, we will take spike counts for each neuron within a single 200ms bin starting 150ms after the reach target turns on. ***

In other words, to calculate firing rates, you will calculate it over the 200ms window: 

`data['train_trial'][n,k][1][i,350:550]`

In [1]:
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math

data = sio.loadmat('ps4_realdata.mat') # load the .mat file.
NumTrainData = data['train_trial'].shape[0]
NumClass = data['train_trial'].shape[1]
NumTestData = data['test_trial'].shape[0]

# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2

### (a) (8 points) 
Fit the ML parameters of model i) to the training data (91 × 8 observations of a length 97 array of neuron firing rates). 

To calculate the firing rates, use a single 200ms bin starting from 150ms after the target turns on.  This corresponds to using `data['train_trial'][n,k][1][i, 350:550]` to calculate all firing rates.  This corresponds to a 200ms window that turns on 150ms after the reach turns on.

Then, use these parameters to classify the test data (91 × 8 data points) according to the decision rule (1). What is the percent of test data points correctly classified?

In [40]:
##4a

# Calculate the firing rates.

trainDataArr =  np.zeros((NumClass,NumTrainData,97)) # contains the firing rates for all neurons on all 8 x 91 trials in the training set
testDataArr =  np.zeros((NumClass,NumTestData,97)) # for the testing set.

for classIX in range(NumClass):
    for trainDataIX in range(NumTrainData):   
        trainDataArr[classIX,trainDataIX,:] = np.sum(data['train_trial'][trainDataIX,classIX][1][:,350:550],1)
    for testDataIX in range(NumTestData):        
        testDataArr[classIX,testDataIX,:]=np.sum(data['test_trial'][testDataIX,classIX][1][:,350:550],1)
#====================================================#
# YOUR CODE HERE:
#   Fit the ML parameters of model i) to training data
#====================================================#



modParam1 = dict()
modParam1['pi'] = np.array([1.0/NumClass] * NumClass)

modParam1['mean'] = np.zeros((NumClass, 97))
for classIX in range(NumClass):
    modParam1['mean'][classIX] = np.mean(trainDataArr[classIX], axis=0)
    
S_tot = np.zeros((97,97))
for classIX in range(NumClass):
    for dataIX in range(NumTrainData):
        miu = modParam1['mean'][classIX].reshape(-1,1)
        X = trainDataArr[classIX, dataIX].reshape(-1,1)
        
        S_tot += np.dot(X - miu, np.transpose(X - miu))

modParam1['cov'] = S_tot/(NumClass * NumTrainData)


#====================================================#
# END YOUR CODE
#====================================================# 

#====================================================#
# YOUR CODE HERE:
#   Classify the test data and print the accuracy
#====================================================#

def mod1_predict(X):
    pred_probs = np.zeros(NumClass)
    for classIX in range(NumClass):
        pi = modParam1['pi'][classIX]
        mean = modParam1['mean'][classIX, :].reshape(-1,1)
        cov=modParam1['cov']

        
        pred_prob = np.log(pi)\
                    + np.dot(np.dot(np.transpose(mean), np.linalg.inv(cov)), X)\
                    - 1/2 * np.dot(np.dot(np.transpose(mean), np.linalg.inv(cov)), 
                                   mean)
        pred_probs[classIX] = pred_prob[0,0]
    return np.argmax(pred_probs)
    
correct_pred_cnt = 0
for classIX in range(NumClass):
    for dataIX in range(NumTestData):
        X = testDataArr[classIX, dataIX].reshape(-1,1)
        pred_class = mod1_predict(X)
        
        if pred_class == classIX:
            correct_pred_cnt += 1
print(correct_pred_cnt/(NumClass*NumTestData))
        

        
#====================================================#
# END YOUR CODE
#====================================================# 


0.9601648351648352


#### Question:
What is the percent of test data points correctly classified?

#### Your answer: 
0.9601648351648352

### (b) (6 points) 
Repeat part (a) for model ii). You `should encounter a Python error` when classifying the test data. What is this error? Why did the Python error occur? What would we need to do to correct this error?

To be concrete, the output of this cell should be a `Python error` and that's all fine.  But we want you to understand what the error is so we can fix it later.


In [49]:
##4b

#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters of model ii) to training data
#====================================================#
# modParam2
modParam2 = dict()
modParam2['pi'] = np.array([1/NumClass] * NumClass)

modParam2['mean'] = np.zeros((NumClass, 97))
for classIX in range(NumClass):
    modParam2['mean'][classIX] = np.mean(trainDataArr[classIX], axis=0)


modParam2['cov'] = np.zeros((NumClass, 97, 97))
for classIX in range(NumClass):
    S = np.zeros((97, 97))
    for dataIX in range(NumTrainData):
        miu = modParam2['mean'][classIX].reshape(-1, 1)
        X = trainDataArr[classIX, dataIX].reshape(-1, 1)
        S += np.dot(X-miu, np.transpose(X-miu))
    S = S/NumTrainData
    modParam2['cov'][classIX] = S
    
    

def mod2_predict(X):
    pred_probs = np.zeros(NumClass)
    for classIX in range(NumClass):
        pi = modParam2['pi'][classIX]
        mean = modParam2['mean'][classIX, :].reshape(-1,1)
        cov=modParam2['cov'][classIX]
        pred_prob = np.log(pi) - 1/2 * np.log(np.linalg.det(cov))\
                    - 1/2 * np.matmul(np.matmul(np.transpose(X-mean), 
                                                np.linalg.inv(cov)), 
                                      X-mean)
        pred_probs[classIX] = pred_prob[0,0]
    return np.argmax(pred_probs)

correct_pred_cnt = 0
for classIX in range(NumClass):
    for dataIX in range(NumTestData):
        X = testDataArr[classIX, dataIX].reshape(-1,1)
        pred_class = mod2_predict(X)
        
        if pred_class == classIX:
            correct_pred_cnt += 1
print(correct_pred_cnt/(NumClass*NumTestData))
#====================================================#
# END YOUR CODE
#====================================================# 


/Users/zxxia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log


LinAlgError: Singular matrix

#### Question:
Why did the python error occur? What would we need to do to correct this error?

#### Your answer:  
Because covariance matrix signma is singular, it has zero determinant and no inverse.

### (c) (8 points) 
Correct the problem from part (b) by detecting and then removing offending neurons that cause the error. Now, what is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

In [77]:
##4c
neuronsToRemove = []
#====================================================#
# YOUR CODE HERE:
#   Detect and then remove the offending neurons, so that 
#   you no longer run into the bug in part (b).
#====================================================#
for classIX in range(NumClass):
    off_neurons_indices = (np.sum(trainDataArr[classIX,:,:].T, 1) == 0).nonzero()
    for idx in off_neurons_indices[0].tolist():
        if idx not in neuronsToRemove:
            neuronsToRemove.append(idx)


trainDataArr_new = np.delete(trainDataArr, neuronsToRemove, axis=2)
testDataArr_new = np.delete(testDataArr, neuronsToRemove, axis=2)


#====================================================#
# END YOUR CODE
#====================================================# 
##
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#


# modParam2
modParam2 = dict()
modParam2['pi'] = np.array([1/NumClass] * NumClass)

modParam2['mean'] = np.zeros((NumClass, trainDataArr_new.shape[2]))
for classIX in range(NumClass):
    modParam2['mean'][classIX] = np.mean(trainDataArr_new[classIX], axis=0)


modParam2['cov'] = np.zeros((NumClass, trainDataArr_new.shape[2], 
                             trainDataArr_new.shape[2]))
for classIX in range(NumClass):
    S = np.zeros((trainDataArr_new.shape[2], trainDataArr_new.shape[2]))
    for dataIX in range(NumTrainData):
        miu = modParam2['mean'][classIX].reshape(-1, 1)
        X = trainDataArr_new[classIX, dataIX].reshape(-1, 1)
        S += np.dot(X-miu, np.transpose(X-miu))
    S = S/NumTrainData
    modParam2['cov'][classIX] = S
    
    

def mod2_predict(X):
    pred_probs = np.zeros(NumClass)
    for classIX in range(NumClass):
        pi = modParam2['pi'][classIX]
        mean = modParam2['mean'][classIX, :].reshape(-1,1)
        cov=modParam2['cov'][classIX]
        pred_prob = np.log(pi) - 1/2 * np.log(np.linalg.det(cov))\
                    - 1/2 * np.matmul(np.matmul(np.transpose(X-mean), 
                                                np.linalg.inv(cov)), X-mean)
        pred_probs[classIX] = pred_prob[0,0]
    return np.argmax(pred_probs)

correct_pred_cnt = 0
for classIX in range(NumClass):
    for dataIX in range(NumTestData):
        X = testDataArr_new[classIX, dataIX].reshape(-1,1)
        pred_class = mod2_predict(X)
        
        if pred_class == classIX:
            correct_pred_cnt += 1
print(correct_pred_cnt/(NumClass*NumTestData))
#====================================================#
# END YOUR CODE
#====================================================# 

0.4409340659340659


#### Question:
What is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

#### Your answer:  
0.4409340659340659.
It is lower than the answer to part(a). Because we lost training information after removing offending neurons.

### (d) (8 points) 
Now we classify using a naive Bayes model. Repeat part (a) for model iii). Keep the convention in part (c), where offending neurons were removed from the anal- ysis.

In [79]:
##4d
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#

modParam3 = dict()
modParam3['pi'] = np.array([1/NumClass] * NumClass)
modParam3['mean'] = np.zeros((NumClass, trainDataArr_new.shape[2]))
for classIX in range(NumClass):
    modParam3['mean'][classIX] = np.mean(trainDataArr_new[classIX], axis=0)

    
def mod3_predict(X):
    pred_probs = np.zeros(NumClass)
    for classIX in range(NumClass):
        pi = modParam3['pi'][classIX]
        mean = modParam3['mean'][classIX, :].reshape(-1,1)
        pred_prob = np.log(pi)\
                    +np.sum(X*np.log(mean)-mean-np.log(scipy.special.factorial(X)), 
                            0).reshape(1,1)
        pred_probs[classIX] = pred_prob[0,0]
    return np.argmax(pred_probs)

correct_pred_cnt = 0
for classIX in range(NumClass):
    for dataIX in range(NumTestData):
        X = testDataArr_new[classIX, dataIX].reshape(-1,1)
        pred_class = mod3_predict(X)
        
        if pred_class == classIX:
            correct_pred_cnt += 1
print(correct_pred_cnt/(NumClass*NumTestData))
#====================================================#
# END YOUR CODE
#====================================================# 

0.9203296703296703


#### Question:
what is the percent of test data points correctly classified? 

#### Your answer: 
0.9203296703296703